In [53]:
# 合并原图、和bbox，labels的数据

In [1]:
import glob
import pandas as pd
from pathlib import Path
import re

### API

In [113]:
def get_list2dict(k_list, v_list):
    d = {}
    for item in zip(k_list, v_list):
        d[item[0]] = item[1]
    return d

### 加载数据

In [ ]:
# 合并

In [ ]:
# !mkdir /nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/info/

In [48]:
!ls /nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/info/

crops_imgs1m_list.json	dadetv4		    rewrite_labels_plist.json
crops_rev_list.json	imgs_rev_list.json


In [83]:
!tree -L 1 /nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/info/dadetv4/

/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/info/dadetv4/
├── bboxes_all_df.csv
├── crops_all_df.csv
├── imgs_det_df.csv
├── imgs_org_df.csv
└── labels_df.csv

0 directories, 5 files


In [49]:
img_org_df = pd.read_csv('/nas/chenyi/datasets_cls/gallery_detect/csv/gall_df.csv')

In [43]:
bboxes_all_df = pd.read_csv('/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/info/dadetv4/bboxes_all_df.csv')

In [45]:
crops_df = pd.read_csv('/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/info/dadetv4/crops_all_df.csv')

In [46]:
len(img_df), len(bboxes_all_df),len(crops_df)

(1661397, 6605289, 6605629)

In [ ]:
# img_org_df.to_csv('/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/info/dadetv4/imgs_org_df.csv', index=False)

In [90]:
bboxes_all_df.to_csv('/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/info/dadetv4/bboxes_all_df.csv', 
                     index=False)

### 构建原图信息

In [58]:
img_df.tail(1).values

array([['/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/gall_dadetv4_all/imgs_det/ffffdea9-74ea-4fdf-b86e-6e77bb7d7979-494872.jpg',
        'ffffdea9-74ea-4fdf-b86e-6e77bb7d7979-494872', 690, 856]],
      dtype=object)

In [59]:
img_org_df.tail(1).values

array([['/nas/oss/gallery/ff/ff/ffffdea9-74ea-4fdf-b86e-6e77bb7d7979-494872.jpg',
        'ffffdea9-74ea-4fdf-b86e-6e77bb7d7979-494872']], dtype=object)

In [ ]:
img_org_df

In [54]:
len(img_df), len(img_org_df)

(1661397, 1661397)

In [55]:
img_org_df.sort_values('stem', inplace=True)
img_df.sort_values('stem', inplace=True)

In [61]:
img_org_df['width'] = img_df['width'].values

In [63]:
img_org_df['height'] = img_df['height'].values

### Bboxes信息合成

In [73]:
len(bboxes_all_df),len(crops_df)

(6605289, 6605629)

In [104]:
len(label_cross_df)

6599176

In [87]:
# bboxes_all_df.rename(columns={bboxes_all_df.columns[0]:'label_pred',
#                              bboxes_all_df.columns[1]:'bx1',
#                               bboxes_all_df.columns[2]:'bx2',
#                               bboxes_all_df.columns[3]:'bx3',
#                               bboxes_all_df.columns[4]:'bx4',
#                               bboxes_all_df.columns[5]:'conf',
#                               bboxes_all_df.columns[6]:'id_crop',
#                               bboxes_all_df.columns[7]:'label_p',
#                              }, inplace=True)

#### label和crops的交集

In [97]:
crops_df.head(2)

,crops_p,crops_path,stem
0,/nas/chenyi/datasets_cls/gallery_detect/galler...,/nas/chenyi/datasets_cls/gallery_detect/galler...,69313640-af52-465e-88c8-c0a0099ca5c7-699544_0
1,/nas/chenyi/datasets_cls/gallery_detect/galler...,/nas/chenyi/datasets_cls/gallery_detect/galler...,69313640-af52-465e-88c8-c0a0099ca5c7-699544_2


In [93]:
bboxes_all_df.head(1)

,label_pred,bx1,bx2,bx3,bx4,conf,id_crop,label_p,stem
0,0,0.373611,0.942361,0.083333,0.115278,0.880866,0,/nas/chenyi/datasets_cls/gallery_detect/galler...,69313640-af52-465e-88c8-c0a0099ca5c7-699544_0


In [92]:
bboxes_all_df['stem'] = bboxes_all_df.apply(lambda x: Path(x['label_p']).stem + '_' + str(x['id_crop']), axis=1)

In [94]:
bboxes_all_df.head(2)

,label_pred,bx1,bx2,bx3,bx4,conf,id_crop,label_p,stem
0,0,0.373611,0.942361,0.083333,0.115278,0.880866,0,/nas/chenyi/datasets_cls/gallery_detect/galler...,69313640-af52-465e-88c8-c0a0099ca5c7-699544_0
1,5,0.601389,0.402778,0.325000,0.463889,0.911707,1,/nas/chenyi/datasets_cls/gallery_detect/galler...,69313640-af52-465e-88c8-c0a0099ca5c7-699544_1


In [96]:
crops_df['stem'] = crops_df['crops_p'].apply(lambda x: Path(x).stem)

In [102]:
label_cross_df = bboxes_all_df[bboxes_all_df['stem'].isin(crops_df['stem'])]

#### 获取原图的宽高

In [108]:
label_cross_df = pd.read_csv('/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/info/dadetv4/bboxes_detv4_df.csv')

In [125]:
label_cross_df.to_csv('/nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/info/dadetv4/bboxes_detv4_df.csv', 
                     index=False)

In [109]:
label_cross_df['stem'] = label_cross_df['label_p'].apply(lambda x: Path(x).stem)

In [119]:
label_cross_df.head(1)

,label_pred,bx1,bx2,bx3,bx4,conf,id_crop,label_p,stem,oss_sp,width,heigth
0,0,0.373611,0.942361,0.083333,0.115278,0.880866,0,/nas/chenyi/datasets_cls/gallery_detect/galler...,69313640-af52-465e-88c8-c0a0099ca5c7-699544,/nas/oss/gallery/69/31/69313640-af52-465e-88c8...,933,933


In [112]:
img_org_df.head()

,oss_sp,stem,width,height
152500,/nas/oss/gallery/00/00/000000c3-920d-43ea-8ef7...,000000c3-920d-43ea-8ef7-35090987f963-839405,1279,1706
152487,/nas/oss/gallery/00/00/000006d9-d433-4d50-80b1...,000006d9-d433-4d50-80b1-2c88518e9f5e-481515,690,993
152517,/nas/oss/gallery/00/00/0000071d-bc81-11ec-9683...,0000071d-bc81-11ec-9683-50814025d3e9,690,1034
152501,/nas/oss/gallery/00/00/00001746-68dd-470f-bd04...,00001746-68dd-470f-bd04-a45489eb1cae-414619,1440,1754
152512,/nas/oss/gallery/00/00/00002245-90c8-11ec-8ace...,00002245-90c8-11ec-8ace-5cfb3aa6f5e6,500,600


In [114]:
stem2oss_dict = get_list2dict(img_org_df['stem'], img_org_df['oss_sp'])

In [117]:
stem2width_dict = get_list2dict(img_org_df['stem'], img_org_df['width'])
stem2height_dict = get_list2dict(img_org_df['stem'], img_org_df['height'])

In [115]:
label_cross_df['oss_sp'] = label_cross_df['stem'].map(stem2oss_dict)

In [118]:
label_cross_df['width'] = label_cross_df['stem'].map(stem2width_dict)
label_cross_df['heigth'] = label_cross_df['stem'].map(stem2height_dict)

In [124]:
!ls /nas/chenyi/datasets_cls/gallery_detect/gallery_oss/dadetv4/gall_dadetv4_all/crops/

bag  lowerbody	shoe  skirt  upperbody	wholebody


In [121]:
model_label_map = {
    0: 10,  # 鞋子
    1: 8,  # 包包
    2: 4,  # 上装
    3: 5,  # 裤子
    4: 6,  # 裙子
    5: 7,  # 连体装
}